# Getting Started with Google Search as a Tool with Gemini in Vertex AI

         

## Overview



[Grounding in Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/ground-gemini) lets you use generative text models to generate content grounded in your own documents and data. This capability lets the model access information at runtime that goes beyond its training data. By grounding model responses in Google Search results or data stores within [Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/enterprise-search-introduction), LLMs that are grounded in data can produce more accurate, up-to-date, and relevant responses.

Grounding provides the following benefits:

- Reduces model hallucinations (instances where the model generates content that isn't factual)
- Anchors model responses to specific information, documents, and data sources
- Enhances the trustworthiness, accuracy, and applicability of the generated content

You can configure two different sources of grounding in Vertex AI:

1. Google Search results for data that is publicly available and indexed.
   - If you use this service in a production application, you will also need to [use a Google Search entry point](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/grounding-search-entry-points).
2. [Data stores in Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest), which can include your own data in the form of website data, unstructured data, or structured data

### Objective

In this tutorial, you learn how to:

- Generate LLM text and chat model responses grounded in Google Search results
- Compare the results of ungrounded LLM responses with grounded LLM responses
- Create and use a data store in Vertex AI Search to ground responses in custom documents and data
- Generate LLM text and chat model responses grounded in Vertex AI Search results

This tutorial uses the following Google Cloud AI services and resources:

- Vertex AI
- Vertex AI Search and Conversation

The steps performed include:

- Configuring the LLM and prompt for various examples
- Sending example prompts to generative text and chat models in Vertex AI
- Setting up a data store in Vertex AI Search with your own data
- Sending example prompts with various levels of grounding (no grounding, web grounding, data store grounding)

### Install Google Gen AI SDK for Python

Install the following packages required to execute this notebook.

In [ ]:
%pip install --upgrade --user --quiet google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.3/196.3 kB 6.2 MB/s eta 0:00:00


Restart the kernel after installing packages:

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your Google Cloud account

If you are running this notebook on Google Colab, you will need to authenticate your environment. To do this, run the new cell below. This step is not required if you are using Vertex AI Workbench.

In [ ]:
import sys

if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
import os

from google import genai

PROJECT_ID = "qwiklabs-asl-00-b36be5350648"  # @param {type: "string"}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Import libraries

In [ ]:
from IPython.display import Markdown, display
from google.genai.types import (
    GenerateContentConfig,
    GoogleSearch,
    Retrieval,
    Tool,
    VertexAISearch,
)

Initialize the Gemini model from Vertex AI:

In [ ]:
MODEL_ID = "gemini-2.0-flash-001"  # @param {type: "string"}

## Example: Grounding with Google Search results

In this example, you'll compare LLM responses with no grounding with responses that are grounded in the results of a Google Search. You'll ask a question about a the most recent solar eclipse.

In [ ]:
PROMPT = "You are an expert in astronomy. When is the next solar eclipse in the US?"

### Text generation without grounding

Make a prediction request to the LLM with no grounding:

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
)

display(Markdown(response.text))

Alright, let's talk solar eclipses!

The next solar eclipse visible in the United States will be an **annular solar eclipse on October 14, 2023**.

Here's a little more information about it:

*   **Type:** Annular. This means the Moon will pass in front of the Sun, but it won't completely cover it. Instead, a bright ring of sunlight will be visible around the Moon's silhouette – often referred to as a "ring of fire".

*   **Path:** The path of annularity (where the ring of fire is visible) will stretch across parts of the western United States, including:

    *   Oregon
    *   California
    *   Nevada
    *   Utah
    *   Arizona
    *   Colorado
    *   New Mexico
    *   Texas

*   **Partial Eclipse:** A partial solar eclipse will be visible across a much wider area of North America, including the rest of the US. This means people outside the path of annularity will still see the Moon cover a portion of the Sun.

**Important Safety Note:** Never look directly at the Sun without proper eye protection during a solar eclipse. Use certified solar viewing glasses (eclipse glasses) or a handheld solar viewer that meets the ISO 12312-2 international safety standard. Sunglasses are not safe for viewing a solar eclipse.

Do you have any other questions about solar eclipses or astronomy in general? I'd be happy to answer them!


### Text generation grounded in Google Search results

You can add the `tools` keyword argument with a `Tool` including `GoogleSearch` to instruct Gemini to first perform a Google Search with the prompt, then construct an answer based on the web search results.

The search queries and [Search Entry Point](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/grounding-search-entry-points) are available for each `Candidate` in the response.

In [ ]:
google_search_tool = Tool(google_search=GoogleSearch())

response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
    config=GenerateContentConfig(tools=[google_search_tool]),
)

display(Markdown(response.text))

Okay, I can help you with that. To provide you with the most accurate information about the next solar eclipse in the US, I will search for it.

The next solar eclipse in the US will be a partial solar eclipse on August 12, 2026. If you're looking for a total solar eclipse, the next one visible in the US will be on March 30, 2033, but it will only be visible in Alaska. For a total solar eclipse in the contiguous United States, you'll have to wait until August 23, 2044.


Note that the response without grounding only has limited information from the LLM about solar eclipses. Whereas the response that was grounded in web search results contains the most up to date information from web search results that are returned as part of the LLM with grounding request.

## Example: Grounding with custom documents and data

In this example, you'll compare LLM responses with no grounding with responses that are grounded in the [results of a data store in Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest).

The data store will contain internal documents from a fictional bank, Cymbal Bank. These documents aren't available on the public internet, so the Gemini model won't have any information about them by default.

### Creating a data store in Vertex AI Search

In this example, you'll use a Google Cloud Storage bucket with a few sample internal documents for our bank. There's some docs about booking business travel, strategic plan for this Fiscal Year and HR docs describing the different jobs available in the company.

Follow the tutorial steps in the Vertex AI Search documentation to:

1. [Create a data store with unstructured data](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#unstructured-data) that loads in documents from the GCS folder `gs://cloud-samples-data/gen-app-builder/search/cymbal-bank-employee`.
2. [Create a search app](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_search_app) that is attached to that data store. You should also enable the **Enterprise edition features** so that you can search indexed records within the data store.

Note: The data store must be in the same project that you are using for Gemini.

You can also follow this notebook to do it with code. [Create a Vertex AI Search Datastore and App
](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/create_datastore_and_search.ipynb)

Once you've created a data store, obtain the Data Store ID and input it below.

Note: You will need to wait for data ingestion to finish before using a data store with grounding. For more information, see [create a data store](https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es).

In [ ]:

DATA_STORE_PROJECT_ID = PROJECT_ID  # @param {type:"string"}
DATA_STORE_REGION = "global"  # @param {type:"string"}
# Replace this with your data store ID from Vertex AI Search
DATA_STORE_ID = "test_1747722446623"  # @param {type:"string"}

DATA_STORE_NAME = f"projects/{PROJECT_ID}/locations/{DATA_STORE_REGION}/collections/default_collection/dataStores/{DATA_STORE_ID}"

Now you can ask a question about the company culture:

In [ ]:
PROMPT = "What is the company culture like?"

### Text generation without grounding

Make a prediction request to the LLM with no grounding:

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
)

display(Markdown(response.text))

To give you the best answer about a company's culture, I need more information. Tell me:

*   **Which company are you asking about?** Please provide the company name.
*   **If possible, tell me the specific location or team you're interested in.** Company culture can vary greatly between different offices or departments.

Once I know the specific company, I can start to gather information from a variety of sources, such as:

*   **Company Website:** Look for mission statements, values, and "about us" pages.
*   **Employee Reviews:** Sites like Glassdoor, Indeed, and Comparably often have employee reviews that discuss company culture.
*   **News Articles and Press Releases:** See how the company portrays itself in the media.
*   **Social Media:** Check the company's LinkedIn, Twitter, Facebook, and Instagram pages to get a sense of their tone and values.
*   **Industry Reports:** Sometimes, industry-specific reports will comment on company culture.
*   **Personal Network:** If you know anyone who works (or has worked) at the company, reach out and ask them about their experiences.

If you can give me the company name, I can provide a much more helpful and specific answer.

### Text generation grounded in Vertex AI Search results

Now we can add the `tools` keyword arg with a grounding tool of `grounding.VertexAISearch()` to instruct the LLM to first perform a search within your custom data store, then construct an answer based on the relevant documents:

In [ ]:
vertex_ai_search_tool = Tool(
    retrieval=Retrieval(vertex_ai_search=VertexAISearch(datastore=DATA_STORE_NAME))
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents="What is the company culture like?",
    config=GenerateContentConfig(tools=[vertex_ai_search_tool]),
)

display(Markdown(response.text))

print(response.candidates[0].grounding_metadata)

Cymbal Bank has a fast-paced, results-oriented company culture. They value:

*   Collaboration
*   Communication
*   Teamwork
*   Improvement and innovation

Cymbal Bank also strives to have an inclusive and diverse workforce, encouraging employees to be open-minded, respectful, inclusive and to be themselves.

grounding_chunks=[GroundingChunk(retrieved_context=GroundingChunkRetrievedContext(text='6. Beyond your first year\nAfter your first year at Cymbal Bank, you will have the opportunity to continue to grow and develop. You will have\nthe opportunity to take on new challenges and to take on new roles. You will also have the opportunity to learn\nfrom your colleagues and to mentor them. 7. Resources\nIf you have any questions or concerns, please do not hesitate to reach out to your manager, HR department, or\nteam members. We are here to help you succeed. 8. Welcome to the team! We are excited to have you on our team and we look forward to working with you. Tools to get familiar with Cymbal Drive: Our internal communication tool. Cymbal Expense: Our expense tracking tool. Cymbal Fly: Our travel booking tool. Cymbal Stay: Our hotel booking tool. Cymbal Work: Our project management tool. Company culture\nCymbal Bank is a fast-paced, results-oriented company. We are always looking for ways to 

Note that the response without grounding doesn't have any context about what company we are asking about. Whereas the response that was grounded in Vertex AI Search results contains information from the documents provided, along with citations of the information.

<div class="alert alert-block alert-warning">
<b>⚠️ Important notes:</b><br>
<br>
<b>If you get an error when running the previous cell:</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;In order for this sample notebook to work with data store in Vertex AI Search,<br>
&nbsp;&nbsp;&nbsp;&nbsp;you'll need to create a <a href="https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_data_store">data store</a> <b>and</b> a <a href="https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_search_app">search app</a> associated with it in Vertex AI Search.<br>
&nbsp;&nbsp;&nbsp;&nbsp;If you only create a data store, the previous request will return errors when making queries against the data store.
<br><br>
<b>If you get an empty response when running the previous cell:</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;You will need to wait for data ingestion to finish before using a data store with grounding.<br>
&nbsp;&nbsp;&nbsp;&nbsp;For more information, see <a href="https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es">create a data store</a>.
</div>
</div>

## Example: Grounded chat responses

You can also use grounding when working with chat models in Vertex AI. In this example, you'll compare LLM responses with no grounding with responses that are grounded in the results of a Google Search and a data store in Vertex AI Search.

In [ ]:
PROMPT = "What are managed datasets in Vertex AI?"
PROMPT_FOLLOWUP = "What types of data can I use?"

### Chat session grounded in Google Search results

Now you can add the `tools` keyword arg with a Tool of `GoogleSearch` to instruct the chat model to first perform a Google Search with the prompt, then construct an answer based on the web search results:

In [ ]:
chat = client.chats.create(
    model=MODEL_ID,
    config=GenerateContentConfig(tools=[Tool(google_search=GoogleSearch())]),
)

response = chat.send_message(PROMPT)
display(Markdown(response.text))

response = chat.send_message(PROMPT_FOLLOWUP)
display(Markdown(response.text))

In Vertex AI, managed datasets serve as a centralized repository designed for machine learning data, offering a structured approach to manage, prepare, and access data for training and serving models.

Here's a breakdown of the key aspects:

*   **Centralized Management:** They provide a single location within Vertex AI for storing and managing your machine learning data.
*   **Data Preparation:** They support various data preparation tasks like cleaning, transformation, and encoding.
*   **Integrated Model Training:** They seamlessly integrate into model training pipelines within Vertex AI, allowing direct utilization of datasets for training without extra setup.
*   **Versioning:** Vertex AI datasets enhance governance through intrinsic versioning.
*   **Access Control:** They offer cross-team access controls.
*   **Data Splitting:** Managed datasets can automatically split data into training, test, and validation sets.
*   **Data Lineage Tracking**: They help track lineage to models for governance and iterative development.
*   **Performance Comparison**: They enable the comparison of AutoML and custom model performance using the same datasets.

To use managed datasets in Vertex AI:

1.  **Create a Dataset:** Go to the Datasets section in Vertex AI and click "Create Dataset." Choose the appropriate data type (Tabular, Image, Text, or Video) and name the dataset.
2.  **Choose a Storage Location:** Specify where to store the dataset. You can import data from local files, Cloud Storage buckets, or BigQuery tables.
3.  **Define Data Schema (Optional):** For tabular datasets, define the schema, including column names and data types.

Vertex AI passes metadata about your dataset to your training application by setting environment variables in your training container. Examples of such variables include: `AIP_DATA_FORMAT`, `AIP_TRAINING_DATA_URI`, `AIP_VALIDATION_DATA_URI`, and `AIP_TEST_DATA_URI`.


Vertex AI supports a wide range of data types for machine learning tasks:

*   **Tabular Datasets:** These consist of structured data organized in rows and columns, typically in CSV or JSON format. They are suitable for classification, regression, and recommendation tasks.
*   **Image Datasets:** These contain images and are used for tasks like image classification, object detection, and image segmentation. Supported image formats include JPEG, GIF, PNG, BMP, and ICO.
*   **Text Datasets:** These consist of written text and are used for natural language processing tasks like text classification, sentiment analysis, and text generation.
*   **Video Datasets:** These contain video data and can be used for video classification, object tracking, and action recognition. Video input can include frames-only or frames with audio.
*   **Audio Datasets:** Audio data can be used in multimodal prompts for generating text.
*   **Documents:** PDF documents are supported and treated as images, where each page is considered one image.

Vertex AI also supports various data transformations, including:

*   **Categorical:** For categorical features.
*   **Numeric:** For numerical features.
*   **Text:** For text-based features.
*   **Timestamp:** For date and time features.

You can import data from various sources, including local files, Cloud Storage buckets, and BigQuery tables.


### Chat session grounded in Vertex AI Search results

Now we can add the `tools` keyword arg with a grounding tool of `grounding.VertexAISearch()` to instruct the chat model to first perform a search within your custom data store, then construct an answer based on the relevant documents:

In [ ]:
PROMPT = "How do I book business travel?"
PROMPT_FOLLOWUP = "Give me more details."

In [ ]:
chat = client.chats.create(
    model=MODEL_ID,
    config=GenerateContentConfig(
        tools=[
            Tool(
                retrieval=Retrieval(
                    vertex_ai_search=VertexAISearch(datastore=DATA_STORE_NAME)
                )
            )
        ]
    ),
)

response = chat.send_message(PROMPT)
display(Markdown(response.text))

response = chat.send_message(PROMPT_FOLLOWUP)
display(Markdown(response.text))

The best way to book business travel generally involves these steps:

1.  **Check Company Policy:** See if your company has a preferred booking tool, specific travel agencies they work with, or any guidelines you need to follow. Some companies have very specific rules about booking the lowest fares, using certain vendors, or pre-approvals.

2.  **Use Company Travel Tools/Agencies:** If your company has designated tools or agencies, use them. These are often set up to ensure compliance with travel policies and can streamline expense reporting. For example, the Cymbal Bank Business Travel Guide suggests using tools like "Cymbal Fly" for flights, "Cymbal Stay" for hotels, and "Cymbal Drive" for rental cars.

3.  **Book Flights, Hotels, and Rental Cars:** Whether through a company tool or on your own (if allowed), book your flights, hotels, and any rental cars you might need. When booking, keep in mind factors such as:

    *   **Location:** Choose hotels close to your meeting locations or with good transportation access.
    *   **Amenities:** Consider if you need amenities like Wi-Fi, a business center, or free breakfast.
    *   **Loyalty Programs:** Use your loyalty program accounts to earn points and miles.


Okay, let's break down the business travel booking process into greater detail, covering key considerations and steps:

**1. Understanding and Adhering to Company Travel Policy**

*   **Review the Policy:** Obtain and thoroughly read your company's travel policy. This document is your guide and outlines what is expected of you when booking travel.
*   **Key Elements:** Pay close attention to aspects such as:
    *   **Approved Booking Channels:** Are you required to use a specific online booking tool, travel agency, or internal travel department?
    *   **Preferred Vendors:** Does the company have negotiated rates or partnerships with specific airlines, hotels, or car rental companies?
    *   **Spending Limits:** Are there maximum amounts you can spend on flights, hotels, meals, or other travel expenses?
    *   ** класу Travel Classes:** Are there restrictions on booking business or first-class flights or suites?
    *   **Pre-Approval Requirements:** Do you need to obtain approval from your manager or a designated approver before booking travel?
    *   **Documentation:** What receipts and documentation are required for expense reimbursement?

**2. Choosing Your Booking Method**

*   **Company-Designated Tools:** If your company mandates the use of a specific online booking tool (like Concur, Egencia, or a custom platform), start there. These tools are pre-configured to enforce company policy and may offer negotiated rates.
*   **Travel Agency:** Some companies prefer you book through a travel agency. The agency can handle all aspects of your travel arrangements, ensure policy compliance, and provide support in case of issues.
*   **Self-Booking (If Allowed):** If your company allows self-booking, you have more flexibility. However, you are also fully responsible for adhering to company policy. You can use airline/hotel websites, online travel agencies (OTAs) like Expedia or Booking.com, or metasearch engines.

**3. Booking Flights**

*   **Consider Flight Times and Connections:** Choose flights that align with your meeting schedule and minimize travel time. Factor in layovers, potential delays, and time zone changes.
*   **Compare Prices:** Use multiple sources to compare flight prices. Check airline websites, OTAs, and metasearch engines (like Google Flights or Kayak).
*   **Airline Loyalty Programs:** If you are not restricted to a specific airline, book with airlines where you have frequent flyer status to earn points and potentially receive upgrades.
*   **Seat Selection:** Select your seat in advance, if possible. Consider factors like legroom, proximity to restrooms, and aisle vs. window preferences.
*   **Check Baggage Allowance:** Be aware of baggage allowance restrictions and fees, especially if you plan to check luggage.

**4. Booking Hotels**

*   **Location, Location, Location:** Choose a hotel that is conveniently located near your meeting location, client offices, or public transportation. Consider the commute time, especially during peak hours.
*   **Business Amenities:** Ensure the hotel offers amenities that are important for business travelers, such as free Wi-Fi, a business center, meeting rooms, early check-in/late check-out options (if needed), and a comfortable workspace in your room.
*   **Hotel Loyalty Programs:** Book with hotels where you have loyalty status to earn points and receive perks like free upgrades, breakfast, or Wi-Fi.
*   **Read Reviews:** Check online reviews from other business travelers to get an idea of the hotel's quality, service, and suitability for business trips.
*   **Negotiated Rates:** Inquire if your company has negotiated rates with specific hotels in the area.

**5. Booking Rental Cars (If Necessary)**

*   **Do You Need a Car?** Consider whether you truly need a rental car. Public transportation, ride-sharing services (like Uber or Lyft), or taxis may be more cost-effective and convenient in some cities.
*   **Rental Car Companies:** Check prices from multiple rental car companies.
*   **Insurance:** Understand the insurance coverage provided by the rental car company and your personal auto insurance policy. You may also have coverage through your credit card.
*   **Size and Type of Vehicle:** Choose a vehicle that is appropriate for your needs. A compact car may be sufficient for solo travel in a city, while a larger vehicle may be needed for transporting colleagues or equipment.
*   **Location:** Pick up and return the car at a convenient location, such as the airport or near your hotel.

**6. Expense Reporting**

*   **Keep All Receipts:** Save all receipts for flights, hotels, rental cars, meals, and other travel expenses.
*   **Expense Report Software:** Utilize your company's expense reporting software (like Concur, Expensify, or similar).
*   **Policy Compliance:** Ensure all expenses comply with the company's travel policy.
*   **Timely Submission:** Submit your expense report promptly to receive reimbursement.

To get the most relevant information, I recommend checking your company's internal resources for specific policies and procedures.


## Cleaning up

To avoid incurring charges to your Google Cloud account for the resources used in this notebook, follow these steps:

1. To avoid unnecessary Google Cloud charges, use the [Google Cloud console](https://console.cloud.google.com/) to delete your project if you do not need it. Learn more in the Google Cloud documentation for [managing and deleting your project](https://cloud.google.com/resource-manager/docs/creating-managing-projects).
1. If you used an existing Google Cloud project, delete the resources you created to avoid incurring charges to your account. For more information, refer to the documentation to [Delete data from a data store in Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/delete-datastores), then delete your data store.
1. Disable the [Vertex AI Search and Conversation API](https://console.cloud.google.com/apis/api/discoveryengine.googleapis.com) and [Vertex AI API](https://console.cloud.google.com/apis/api/aiplatform.googleapis.com) in the Google Cloud Console.